In [ ]:
jiant_url = "https://drive.google.com/u/0/uc?id=1lFbvdomHhLNSxm37OBxGYsinWy3vYC9U&export=download"
!wget "$jiant_url" -O jiant.zip
!unzip jiant.zip -d ./
!rm jiant.zip

%cd jiant
!pip install -r requirements-no-torch.txt
!pip install --no-deps -e ./

Restart runtime after installing libs

In [2]:
%cd /content/jiant

import jiant.utils.python.io as py_io
import jiant.utils.display as display
import os

def init_task_config(task_names):
  for task_name in task_names:
    os.makedirs("/content/tasks/configs/", exist_ok=True)
    os.makedirs(f"/content/tasks/data/{task_name}", exist_ok=True)

    py_io.write_json({
      "task": task_name,
      "paths": {
        "train": f"/content/tasks/data/{task_name}/train.jsonl",
        "val":   f"/content/tasks/data/{task_name}/val.jsonl",
      },
      "name": task_name
    }, f"/content/tasks/configs/{task_name}_config.json")

/content/jiant


In [3]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define tasks in jiant 

init_task_config(task_names)

In [4]:
# contains data of all probing tasks above
task_data_url = "https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download"
!wget "$task_data_url" -O /content/task_data.zip
!unzip /content/task_data.zip -d /content/tasks/
!rm /content/task_data.zip

--2021-06-26 11:58:21--  https://drive.google.com/u/0/uc?id=1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f&export=download
Resolving drive.google.com (drive.google.com)... 142.250.101.100, 142.250.101.139, 142.250.101.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.101.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q6enplqvo9haevhfs8u5ptck3v1un0tb/1624708650000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download [following]
--2021-06-26 11:58:22--  https://doc-04-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q6enplqvo9haevhfs8u5ptck3v1un0tb/1624708650000/13675844253040106205/*/1-nlA_dTsAqQ5oO0vii2lMsm34yYMmn_f?e=download
Resolving doc-04-18-docs.googleusercontent.com (doc-04-18-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-04-18-docs.googleusercontent.co

In [5]:
import jiant.proj.main.export_model as export_model

models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]

for model in models:
  export_model.export_model(
      hf_pretrained_model_name_or_path=model,
      output_base_path=f"/content/models/{model}",
  )

Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Some weights of the model checkpoint at csarron/bert-base-uncased-squad-v1 were not used when initializing BertForPreTraining: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at csarron/bert-base-uncased-squad-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.

In [ ]:
import jiant.shared.caching as caching
import jiant.proj.main.tokenize_and_cache as tokenize_and_cache

# Tokenize and cache each task
def tokenize(task_names, models):
  for task_name in task_names:
    for model in models:
      tokenize_and_cache.main(tokenize_and_cache.RunConfiguration(
          task_config_path=f"/content/tasks/configs/{task_name}_config.json",
          hf_pretrained_model_name_or_path=model,
          output_dir=f"/content/cache/{task_name}",
          phases=["train", "val"],
      ))

tokenize(task_names, models)

In [7]:
row = caching.ChunkedFilesDataCache(f"/content/cache/{task_names[3]}/train").load_chunk(0)[0]["data_row"]
print(row.input_ids)
print(row.tokens)
print(row.spans)
print(row.tokens[row.spans[0][0]: row.spans[0][1]+1])
#print(row.tokens[row.spans[1][0]: row.spans[1][1]+1])

[  101  2006  2285  1023  1010  1996  1000 14004  1011 10556 11631  5332
  5575  1000  3864  2234  2000  2019  2203  1012   102     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
['[CLS]', 'on', 'december', '9', ',', 'the', '"', 'taipei', '-', 'ka', '##oh', '##si', '##ung', '"', 'elections', 'came', 'to', 'an', 'end', '.', '[SEP]']
[[15 15]
 [16 18]]
['came']


In [8]:
import jiant.proj.main.scripts.configurator as configurator

def create_jiant_task_config(task_name, epochs=5):
  jiant_run_config = configurator.SimpleAPIMultiTaskConfigurator(
      task_config_base_path="/content/tasks/configs",
      task_cache_base_path="/content/cache",
      train_task_name_list=[task_name],
      val_task_name_list=[task_name],
      train_batch_size=8,
      eval_batch_size=2,
      epochs=epochs,
      num_gpus=1,
  ).create_config()
  os.makedirs("/content/run_configs/", exist_ok=True)
  py_io.write_json(jiant_run_config, f"/content/run_configs/{task_name}_run_config.json")
  #display.show_json(jiant_run_config)

In [9]:
# You may restart runtime when you modify the jiant source code

import jiant.proj.main.runscript as main_runscript

def run_probing_task(task_name, model_name="bert-base-uncased", num_layers=1, eval_every_steps=1000):
  run_args = main_runscript.RunConfiguration(
      jiant_task_container_config_path=f"/content/run_configs/{task_name}_run_config.json",
      output_dir=f"/content/runs/{task_name}",
      hf_pretrained_model_name_or_path=model_name,
      model_path=f"/content/models/{model_name}/model/model.p",
      model_config_path=f"/content/models/{model_name}/model/config.json",
      learning_rate=1e-5,
      eval_every_steps=eval_every_steps,
      do_train=True,
      do_val=True,
      do_save=True,
      force_overwrite=True,
      num_hidden_layers=num_layers,
  )
  return main_runscript.run_loop(run_args)

In [ ]:
task_names = ["ner", "semeval", "coref",  # supported by jiant 
...           "srl"]                      # additional task for other badge     
#...           "sup-squad", "ques-squad"]  # todo: define new tasks in jiant 
models = ["bert-base-uncased", "csarron/bert-base-uncased-squad-v1"]
num_layers = list(range(1, 3))      # from 1 to 12 layers

# Hyperparameters
epochs = 5              
eval_every_steps = 1000

result = "model\ttask\tlayer\tacc\n"

for model in models:
  for task in task_names:
    for n_layers in num_layers:
      create_jiant_task_config(task, epochs=epochs)
      probing_output = run_probing_task(task, model, n_layers, eval_every_steps=eval_every_steps)
      result += model + "\t" + task + "\t" + str(n_layers) + "\t" + str(probing_output[task]["metrics"]["minor"]["acc"]) + "\n"

In [11]:
print(result)

model	task	layer	acc
bert-base-uncased	ner	1	0.9444444444444444
bert-base-uncased	ner	2	0.9444444444444444
bert-base-uncased	semeval	1	0.9210526315789473
bert-base-uncased	semeval	2	0.7631578947368421
bert-base-uncased	coref	1	0.5952380952380952
bert-base-uncased	coref	2	0.5952380952380952
bert-base-uncased	srl	1	0.9204545454545454
bert-base-uncased	srl	2	0.9318181818181818
csarron/bert-base-uncased-squad-v1	ner	1	0.9444444444444444
csarron/bert-base-uncased-squad-v1	ner	2	0.9444444444444444
csarron/bert-base-uncased-squad-v1	semeval	1	0.8947368421052632
csarron/bert-base-uncased-squad-v1	semeval	2	0.8157894736842105
csarron/bert-base-uncased-squad-v1	coref	1	0.5476190476190477
csarron/bert-base-uncased-squad-v1	coref	2	0.7380952380952381
csarron/bert-base-uncased-squad-v1	srl	1	0.9204545454545454
csarron/bert-base-uncased-squad-v1	srl	2	0.9583333333333334

